### Wstęp do Uczenia Maszynowego 
##### Laboratorium 08

In [1]:
# import pakietów
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [2]:
# zbiory danych
X = pd.read_csv("X.csv")
y = pd.read_csv("y.csv")

### Zadanie 1
-----
Przygotuj dane w następujący sposób:

a) Ze zbioru `X` wybierz tylko zmienne `"education", "workclass", "sex", "hours-per-week"`.

b) Dokonaj transformacji dla zmiennej `"education"`

    1, education in ['Masters', 'some-college', 'Bachelors', 'Doctorate']
    0, w przeciwnym przypadku

c) Dokonaj transformacji dla zmiennej `"workclass"`

    1, workclass == 'Private'
    0, w przeciwnym przypadku

d) Dokonaj transformacji dla zmiennej `"sex"`

    1, sex == 'Male
    0, w przeciwnym przypadku

e) Dokonaj transformacji dla zmiennej `"hours-per-week"`

    1, hours-per-week <= 40 
    0, w przeciwnym przypadku

In [3]:
# wybieramy zgodnie z poleceniem 4 kolumny do dalszej obróbki
tmp = X[["education", "workclass", "sex", "hours-per-week"]]
tmp

,education,workclass,sex,hours-per-week
0,Bachelors,State-gov,Male,40
1,Bachelors,Self-emp-not-inc,Male,13
2,HS-grad,Private,Male,40
3,11th,Private,Male,40
4,Bachelors,Private,Female,40
...,...,...,...,...
48837,Bachelors,Private,Female,36
48838,HS-grad,NaN,Male,40
48839,Bachelors,Private,Male,50
48840,Bachelors,Private,Male,40


In [4]:
# tworzenie zmiennych binarnych zgodnie z poleceniem
X = pd.DataFrame({"education" : tmp["education"].isin(["some-college", "Masters", "Bachelors", "Doctorate"]),
                  "workclass" : tmp["workclass"].isin(["Private"]) ,
                  "sex" : tmp["sex"].isin(["Male"]),
                  "hours" : np.where(tmp["hours-per-week"] <= 40, True, False)})

In [5]:
# wektor y przyjmuje 4 wartości
y.value_counts()

income
<=50K     24720
<=50K.    12435
>50K       7841
>50K.      3846
Name: count, dtype: int64

In [6]:
# zamiana wartości z kropką na bez
y.loc[y.income == '<=50K.'] = '<=50K'
y.loc[y.income == '>50K.'] = '>50K'

In [7]:
# dwie klasy
y.value_counts()

income
<=50K     37155
>50K      11687
Name: count, dtype: int64

### 1. Naive Bayes

**Twierdzenie Bayesa**
$$P(A|B) = \frac{P(B|A)\cdot P(A)}{P(B)}$$

$$ $$

Naiwny klasyfikator Bayesa (Naive Bayes) to algorytm uczenia maszynowego oparty na twierdzeniu Bayesa z założeniem naiwnej (uproszczonej) niezależności cech.
Twierdzenie Bayesa określa następującą zależność, biorąc pod uwagę zmienną objaśnianą i wektor cech:

$$P(y | x_1, \dots, x_n) = \frac{P(y)P(x_1, \dots, x_n|y)}{P(x_1, \dots, x_n)}$$

Przyjmując naiwne założenie niezależności warunkowej takiej, że

$$P(x_i|y, x_1, \dots, x_{i-1}, x_{i+1}, \dots, x_n) = P(x_i|y),$$

dla wszystkich $i$ zależność upraszcza się do postaci

$$P(y | x_1, \dots, x_n)  = \frac{P(y)\prod_{i=1}^{n}P(x_i|y)}{P(x_1, \dots, x_n)}.$$

Ponieważ $P(x_1, \dots, x_n)$ jest stałe, możemy zastosować następującą regułę klasyfikacji:
$$P(y | x_1, \dots, x_n) \propto P(y)\prod_{i=1}^{n}P(x_i|y)$$

\begin{equation}
   \hat{y} = \arg \max_{y} P(y)\prod_{i=1}^{n}P(x_i|y)
\end{equation}

### Zadanie 2
-----
Podziel zbiór danych na treningowy i testowy w stosunktu 4:1, ustaw `random_state = 2025`. Wylicz prawdopodobieństwa dla poszczególnych zmiennych pod warunkiem `Y`. Oblicz przynależność do klasy dla pierwszej obserwacji ze zbioru testowego na postawie wyliczonych prawdopodobieństw.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2025)

In [9]:
# wyliczenie P(y)
print('<=50K:', round(np.mean(y_train.income == "<=50K"), 4)) # prawdopodobieństwo klasy <=50K
print('>50K:', round(np.mean(y_train.income != "<=50K"), 4)) # prawdopodobieństwo klasy >50K

<=50K: 0.7612
>50K: 0.2388


In [10]:
# łączymy X_train i y_train, żeby wyliczyć P(x_i|y)
all = pd.concat([X_train, y_train], axis=1)

In [11]:
# P(x_i|y)
print(round(all.groupby(['income'])['education'].value_counts(normalize = True).unstack(), 4))
print(round(all.groupby(['income'])['workclass'].value_counts(normalize = True).unstack(), 4))
print(round(all.groupby(['income'])['sex'].value_counts(normalize = True).unstack(), 4))
print(round(all.groupby(['income'])['hours'].value_counts(normalize = True).unstack(), 4))

education   False   True 
income                   
<=50K      0.8379  0.1621
>50K       0.5543  0.4457
workclass   False   True 
income                   
<=50K      0.2851  0.7149
>50K       0.3643  0.6357
sex      False   True 
income                
<=50K   0.3909  0.6091
>50K    0.1511  0.8489
hours    False   True 
income                
<=50K   0.2304  0.7696
>50K    0.4873  0.5127


In [12]:
# pierwsza obserwacja w zbiorze testowym
X_test.iloc[0,:]

education     True
workclass     True
sex          False
hours        False
Name: 15327, dtype: bool

In [13]:
# wyliczamy predykcję dla y = "<=50K"

# education = True | y = "<=50K" - 0.1621
# workclass = True | y = "<=50K" - 0.7149
# sex = False | y = "<=50K" - 0.3909
# hours = False | y = "<=50K" - 0.2304
# y = "<=50K" - 0.7612

0.1621 * 0.7149 *  0.3909 * 0.2304 * 0.7612

0.007944658552210913

In [14]:
# wyliczamy predykcję dla y = ">50K"

# education = True | y = ">50K" - 0.4457
# workclass = True | y = ">50K" - 0.6357
# sex = False | y = ">50K" - 0.1511
# hours = False | y = ">50K" - 0.4873
# y = ">50K" - 0.2388

0.4457 * 0.6357 * 0.1511 * 0.4873 * 0.2388

0.0049818430783041675

Wniosek: predykcja dla obserwacji to "<=50K"

### Zadanie 3
-----
Przygotuj model Naive Bayes używająć funkcji `BernouliiNB` na danych z Zadania 2. Sprawdź predykcję dla pierwszej obserwacji ze zbioru testowego.

In [15]:
# import modelu Naive Bayes
from sklearn.naive_bayes import BernoulliNB

In [16]:
# trening na danych
NB = BernoulliNB()
NB.fit(X_train, y_train.income)

BernoulliNB()

In [17]:
# predykcja dla zbiorze testowym
pred = NB.predict(X_test)

In [18]:
# predykcja pierwszej obserwacji ze zbioru testowego
pred[0]
# zgadza się z wyliczeniami z Zadania 2

'<=50K'

### 2. Model $k$-najbliższych sąsiadów ($k$-nearest neighborsk, $k$-nn)


Algorytm $k$-najbliższych sąsiadów ($k$-NN) jest prostym, nieparametrycznym, algorytmem "leniwego uczenia" (lazy learning), stosowanym zarówno do zadań klasyfikacji, jak i regresji.

Podstawową ideą $k$-NN jest założenie, że punkty danych znajdujące się blisko siebie w przestrzeni cech prawdopodobnie mają tę samą klasę (w przypadku klasyfikacji). $K$ oznacza liczbę najbliższych sąsiadów (punktów danych) w zbiorze treningowym, które algorytm bierze pod uwagę przy dokonywaniu predykcji dla nowego, nieznanego punktu danych. Wartość $k$ jest zazwyczaj nieparzystą liczbą całkowitą (aby uniknąć remisu w klasyfikacji).

Algorytm oblicza odległość między punktami, powszechnie stosowana metryka odległości to **odległość Euklidesowa**.

### Zadanie 4
-----

a) Podziel losowo zbiór na część treningową i testową (7:3).

b) Przeskaluj zmienne w zbiorze treningowym i testowym za pomocą średniej i odchylenia standardowego danej zmiennej na zbiorze treningowym.

c) Dopasuj metodę k najbliższych sąsiadów dla $k = 5, 10, 20, 50, 80, 100, 150$ na zbiorze treningowym. Rozważ przynajmniej 3-krotną kroswalidację.

d) Sprawdź na zbiorze testowym współczynnik AUC dla wyliczonych $k$ z polecenia c).

e) Wybierz optymalne $k$ bazując na punkcie c) i d). Przygotuj model kNN z jednakową wagą dla wszystkich sąsiadów oraz z wagą uzależnioną od odległości między obserwacją a jej sąsiadem.

In [19]:
df = pd.read_csv("SAheart.data")

In [20]:
# podział zbioru danych na X i y
y = df.chd
X = df.drop(['chd'], axis = 1)
# mamy jedną zmienną kategoryczną - zmiana na numeryczną
X = pd.get_dummies(X, drop_first=True)

# podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# skalowanie danych z wykorzystaniem StandarScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
X_train_std = scaler.fit_transform(X_train) 
X_test_std = scaler.transform(X_test)


In [21]:
# definicja kroswalidacji
import sklearn.model_selection as skm
kfold = skm.KFold(5,
                  random_state=0,
                  shuffle=True)

In [22]:
# import modelu kNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [23]:
# tworzenie siatki hiperparametrów do przeszukiwania - wartości k w modelu kNN
grid = skm.GridSearchCV(knn, # model
                        {'n_neighbors':[5, 10, 20, 50, 80, 100, 150]}, # słownik hiperparametrów
                         refit=True,
                         cv=kfold, # informacje o kroswalidacji zdefiniowane powyżej
                         scoring='roc_auc') # miara pod którą optymalizujemy

In [24]:
grid.fit(X_train_std, y_train) # trening modelu na kroswalidacji i przeszukiwanie hiperparametru

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [5, 10, 20, 50, 80, 100, 150]},
             scoring='roc_auc')

In [25]:
grid.best_params_ # najlepsze wartości hiperparametrów

{'n_neighbors': 150}

In [26]:
# podsumowanie całego eksperymentu (wybranie wartości miary i hiperparametru z kroswaliacji)
pd.DataFrame(grid.cv_results_)[["mean_test_score", "std_test_score", "param_n_neighbors"]]

,mean_test_score,std_test_score,param_n_neighbors
0,0.676461,0.038719,5
1,0.711231,0.050549,10
2,0.737770,0.042770,20
3,0.759064,0.029982,50
4,0.752852,0.036625,80
5,0.761576,0.036527,100
6,0.764130,0.024929,150


In [27]:
# porównanie modeli z ustawionym hiperparametrem 'weights' - który przypisuje wagi (odległość) k najbliższym obserwacjom na których bazuje predykcja 
knn = KNeighborsClassifier(n_neighbors = grid.best_params_["n_neighbors"])
knn_distance = KNeighborsClassifier(n_neighbors = grid.best_params_["n_neighbors"], weights = "distance")

knn.fit(X_train_std, y_train)
knn_distance.fit(X_train_std, y_train)

from sklearn.metrics import roc_auc_score

print("knn: ", round(roc_auc_score(y_test, knn.predict_proba(X_test_std)[:,1]), 4))
print("knn_distance :", round(roc_auc_score(y_test, knn_distance.predict_proba(X_test_std)[:,1]), 4))

knn:  0.7033
knn_distance : 0.7075
